In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import time
from datetime import datetime

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import scipy.stats as stats

import math
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error
from math import sqrt

import torch
import gpytorch
from gpytorch.kernels import RBFKernel as RBF
from gpytorch.kernels import ScaleKernel as C
from gpytorch.kernels import PeriodicKernel as Per
from gpytorch.kernels import RQKernel as RQ
from gpytorch.kernels import MaternKernel as M
from gpytorch.kernels import PolynomialKernel

import properscoring as prscore

## Read and preprocess the dataset

In [ ]:
df = pd.read_csv('power_weather_data.csv')

# csv file MUST contain 'date' and 'Power' fields
# optional: weather data

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y %H:%M')

In [ ]:
df['hour'] = df['date'].apply(lambda x: x.hour )
df['month'] = df['date'].apply(lambda x: x.month)

In [ ]:
# df['hour_sin'] = np.sin(df['hour'] * 2 * np.pi/24)
# df['hour_cos'] = np.cos(df['hour'] * 2 * np.pi/24)
df['month_sin'] = np.sin(df['month'] * 2 * np.pi/12)
df['month_cos'] = np.cos(df['month'] * 2 * np.pi/12)

In [ ]:
df = df[(df['hour']>=6) & (df['hour']<=21)]

In [ ]:
# df = df.drop(['hour', 'month'], axis=1)
df = df.drop(['month'], axis=1)

In [ ]:
P = df['Power']

PowerData = pd.concat([P.shift(3), P.shift(2), P.shift(1)], axis=1)
PowerData.columns = ['t-45', 't-30', 't-15']

df = pd.concat([df, PowerData.reindex(df.index)], axis=1)
    
df = df.fillna(0)

## Hyperparameters

In [ ]:
weeks = [['2018-03-01', '2019-03-15']]

val_days = 14

# n_points_day = 4 * 24
n_points_day = 4 * 16

## Set the dataframes

In [ ]:
dfs = []

for w in weeks:
    
    w_start = datetime.strptime(w[0]+" 00:00", '%Y-%m-%d %H:%M')
    w_end = datetime.strptime(w[1]+" 23:59", '%Y-%m-%d %H:%M')
    
    dfs.append(df[(df['date'] > w_start) & (df['date'] < w_end)])
    
n_sets = len(dfs)

## Train Test Split

In [ ]:
X_train_ = []
X_test_ = []
y_train_ = []
y_test = []

x_scaler = []
y_scaler = []

t_train = []
t_test = []

for i in range(len(dfs)):

    train = dfs[i][:int(-n_points_day*val_days)]
    test = dfs[i][int(-n_points_day*val_days):]
    
    X_tr = train.drop(['Power','date'], axis=1).values
    X_t = test.drop(['Power','date'], axis=1).values
    
    y_tr = train['Power'].values
    y_t = test['Power'].values
    
    x_sc = MinMaxScaler()
    y_sc = MinMaxScaler(feature_range=(-1,1))
#     x_sc = StandardScaler()
#     y_sc = StandardScaler()
    x_sc.fit(X_tr)
    y_sc.fit(y_tr.reshape(-1, 1))
    x_scaler.append(x_sc)
    y_scaler.append(y_sc)
    
    X_train_.append(x_sc.transform(X_tr))
    X_test_.append(x_sc.transform(X_t))
    y_train_.append(y_sc.transform(y_tr.reshape(-1, 1)))
    y_test.append(y_t)
    
    t_train.append(dfs[i].iloc[:int(-n_points_day*val_days)]['date'].values)
    t_test.append(dfs[i].iloc[int(-n_points_day*val_days):]['date'].values)
    

In [ ]:
X_train = []
X_test = []
y_train = []

for i in range(len(dfs)):
    X_train.append(torch.from_numpy(X_train_[i]))
    X_test.append(torch.from_numpy(X_test_[i]))
    
    y_tr = torch.from_numpy(y_train_[i])
    y_train.append(torch.flatten(y_tr))

## GP Model

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, X_train, y_train, likelihood):
        super(ExactGPModel, self).__init__(X_train, y_train, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = C(RQ())    

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
training_iter = 1000
train_loss = []

models = []
likelihoods = []

start = time.time()

for i in range(len(dfs)):
    
    print(i)
    X_tr = X_train[i]
    y_tr = y_train[i]
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(X_tr, y_tr, likelihood)

    model = model.double()
    likelihood = likelihood.double()

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam([{'params': model.parameters()}], lr=0.07) 

    # Loss for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    ite = []
    loss_all = []
    
    for j in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(X_tr)
        # Calculate loss and backprop gradients
        loss = -mll(output, y_tr)
        loss.backward()

        optimizer.step()
        ite = np.append(ite, j)
        loss_all = np.append(loss_all, loss.detach().numpy())
        
    
    train_loss.append(loss_all)
    models.append(model)
    likelihoods.append(likelihood)

    
end = time.time()
print((end - start)/len(dfs))

## Evaluation

In [ ]:
def PICP_func(y, lower, upper):
    sum_points = 0
    for i, yi in enumerate(y):
        if lower[i] <= yi <= upper[i]:
            sum_points += 1
    
    return sum_points / len(y)

def PINAW_func(y, lower, upper):
    PIAW = np.mean(upper - lower)
    R = np.max(y) - np.min(y)
    PINAW = PIAW / R
    
    return PINAW

In [ ]:
for i in range(len(dfs)):
    
    print(i)
    
    # Unpacking
    model = models[i]
    likelihood = likelihoods[i]
    X_t = X_test[i]
    y_t = y_test[i]
    x_sc = x_scaler[i]
    y_sc = y_scaler[i]
    
    
    model.eval()
    likelihood.eval()
    
    # For multi-step ahead prediction
    y_45 = model(X_t[0].unsqueeze(0)).mean
    y_30 = model(X_t[1].unsqueeze(0)).mean
    y_15 = model(X_t[2].unsqueeze(0)).mean
    for j in range(3, X_t.shape[0]):
        X_t[j][-3] = y_45
        X_t[j][-2] = y_30
        X_t[j][-1] = y_15
        y_pred_j = model(X_t[j].unsqueeze(0))
        y_45 = y_30
        y_30 = y_15
        y_15 = y_pred_j.mean
    # end of multi-step ahead
    
    y_pred_i = model(X_t)
    f_pred_i = likelihood(model(X_t))
    
    y_pred = y_pred_i.mean
    y_var = y_pred_i.variance
    y_covar = y_pred_i.covariance_matrix
    
    y_pred = y_pred.detach().numpy()
    
    real_y_pred = y_sc.inverse_transform(y_pred.reshape(-1, 1))
    
    real_y_pred = real_y_pred.flatten()
    real_y_test = y_t.flatten()
    
    lower, upper = f_pred_i.confidence_region()
    
    lower = lower.detach().numpy()
    upper = upper.detach().numpy()
    
    lower = y_sc.inverse_transform(lower.reshape(-1, 1))
    upper = y_sc.inverse_transform(upper.reshape(-1, 1))
    
    lower = lower.flatten()
    upper = upper.flatten()
    
    mean = (upper+lower)/2
    std = (mean - lower)/1.96
    
    # Deterministic metrics
    MAE = mean_absolute_error(real_y_test, mean)
    RMSE = mean_squared_error(real_y_test, mean, squared=False)
    MBE = np.mean(mean - real_y_test)
    print(f'MAE: {MAE:.3f}')
    print(f'RMSE: {RMSE:.3f}')
    print(f'MBE: {MBE:.3f}')
    
    # Probabilistic metrics
    PICP = PICP_func(real_y_test, lower, upper)
    PINAW = PINAW_func(real_y_test, lower, upper)
    C = prscore.crps_gaussian(real_y_test, mu=mean, sig=std)
    CRPS = C.mean()
    print(f'PICP: {PICP:.3f}')
    print(f'PINAW: {PINAW:.3f}')
    print(f'CRPS: {CRPS:.3f}')
    print('\n')